## 1. Importar Librerías

In [ ]:
import asyncio
import os
import random
from dotenv import load_dotenv
from typing import Annotated
from pydantic import Field
from datetime import datetime
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from azure.identity import DefaultAzureCredential
from azure.identity.aio import get_bearer_token_provider

## 2. Configuración

In [ ]:
load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
MODEL = os.getenv("OPENAI_MODEL") or os.getenv("MODEL")

if not AZURE_OPENAI_ENDPOINT or not MODEL:
    raise RuntimeError(
        "Faltan variables de entorno: AZURE_OPENAI_ENDPOINT y/o OPENAI_MODEL"
    )

print(f"✅ Konfiguration geladen: {MODEL}")

## 3. Crear Cliente

In [ ]:
client = OpenAIChatClient(
    base_url=AZURE_OPENAI_ENDPOINT.rstrip("/") + "/openai/v1/",
    api_key=get_bearer_token_provider(
        DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
    ),
    model_id=MODEL,
)

## 4. Herramientas en Alemán - Wochenendplanung

Note que las **descripciones** de los parámetros están en alemán. Esto ayuda al modelo a entender mejor el contexto.

In [ ]:
def get_weather(
    city: Annotated[str, Field(description="Die Stadt, für die das Wetter abgerufen werden soll.")],
    date: Annotated[str, Field(description="Das Datum für die Wettervorhersage im Format YYYY-MM-DD.")],
) -> dict:
    """Holt das Wetter für eine Stadt."""
    print(f"  ☀️ Hole Wetter für {city} am {date}")
    if random.random() < 0.05:
        return {"Temperatur": 72, "Beschreibung": "Sonnig"}
    else:
        return {"Temperatur": 60, "Beschreibung": "Regnerisch"}

def get_activities(
    city: Annotated[str, Field(description="Die Stadt, für die Aktivitäten gesucht werden.")],
    date: Annotated[str, Field(description="Das Datum für die Aktivitäten im Format YYYY-MM-DD.")],
) -> list[dict]:
    """Holt verfügbare Aktivitäten."""
    print(f"  🎯 Hole Aktivitäten für {city} am {date}")
    return [
        {"name": "Wandern", "Standort": city},
        {"name": "Strand", "Standort": city},
        {"name": "Museum", "Standort": city},
    ]

def get_current_date() -> str:
    """Holt das aktuelle Datum."""
    print("  📅 Hole aktuelles Datum")
    return datetime.now().strftime("%Y-%m-%d")

print("✅ Wochenendplanungs-Tools definiert")

## 5. Herramientas en Alemán - Mahlzeitplanung

In [ ]:
def find_recipes(
    query: Annotated[str, Field(description="Benutzeranfrage oder gewünschte Mahlzeit/Zutat")],
) -> list[dict]:
    """Sucht Rezepte basierend auf Anfrage."""
    print(f"  🍳 Suche Rezepte für '{query}'")
    if "pasta" in query.lower():
        recipes = [
            {
                "Titel": "Pasta Primavera",
                "Zutaten": ["Nudeln", "Gemüse", "Olivenöl"],
                "Schritte": ["Nudeln kochen.", "Gemüse anbraten."],
            }
        ]
    elif "tofu" in query.lower():
        recipes = [
            {
                "Titel": "Tofu Pfannengericht",
                "Zutaten": ["Tofu", "Sojasauce", "Gemüse"],
                "Schritte": ["Tofu würfeln.", "Gemüse anbraten."],
            }
        ]
    else:
        recipes = [
            {
                "Titel": "Gegrilltes Käsesandwich",
                "Zutaten": ["Brot", "Käse", "Butter"],
                "Schritte": ["Brot buttern.", "Käse zwischen die Scheiben legen.", "Golden braun grillen."],
            }
        ]
    return recipes

def check_fridge() -> list[str]:
    """Überprüft Zutaten im Kühlschrank."""
    print("  🥗 Überprüfe Kühlschrank auf vorhandene Zutaten")
    if random.random() < 0.5:
        items = ["Nudeln", "Tomatensauce", "Paprika", "Olivenöl"]
    else:
        items = ["Tofu", "Sojasauce", "Brokkoli", "Karotten"]
    return items

print("✅ Mahlzeitplanungs-Tools definiert")

## 6. Crear Agentes Especializados en Alemán

Las instrucciones están completamente en alemán.

In [ ]:
weekend_agent = ChatAgent(
    chat_client=client,
    instructions=(
        "Sie helfen Benutzern bei der Planung ihrer Wochenenden und der Auswahl der besten Aktivitäten für das jeweilige Wetter. "
        "Wenn eine Aktivität bei dem Wetter unangenehm wäre, schlagen Sie sie nicht vor. "
        "Fügen Sie das Datum des Wochenendes in Ihrer Antwort ein."
    ),
    tools=[get_weather, get_activities, get_current_date],
)

meal_agent = ChatAgent(
    chat_client=client,
    instructions=(
        "Sie helfen Benutzern bei der Planung von Mahlzeiten und der Auswahl der besten Rezepte. "
        "Fügen Sie die Zutaten und Kochanweisungen in Ihre Antwort ein. "
        "Geben Sie an, was der Benutzer im Geschäft kaufen muss, wenn Zutaten im Kühlschrank fehlen."
    ),
    tools=[find_recipes, check_fridge],
)

print("✅ Spezialisierte Agenten erstellt")
print("   - Wochenendplanungsagent")
print("   - Mahlzeitplanungsagent")

## 7. Funciones Wrapper

In [ ]:
async def plan_weekend(query: str) -> str:
    """Ruft den Wochenendplanungsagenten auf."""
    print("\n🔧 Tool: Wochenendplanung aufgerufen")
    response = await weekend_agent.run(query)
    return response.text

async def plan_meal(query: str) -> str:
    """Ruft den Mahlzeitplanungsagenten auf."""
    print("\n🔧 Tool: Mahlzeitplanung aufgerufen")
    response = await meal_agent.run(query)
    return response.text

print("✅ Wrapper-Funktionen erstellt")

## 8. Crear Supervisor en Alemán

In [ ]:
supervisor_agent = ChatAgent(
    chat_client=client,
    instructions=(
        "Sie sind ein Supervisor, der zwei Spezialagenten verwaltet: einen Wochenendplanungsagenten und einen Mahlzeitplanungsagenten. "
        "Analysieren Sie die Anfrage des Benutzers, entscheiden Sie, welchen Spezialisten (oder beide) Sie über die verfügbaren Tools aufrufen möchten, "
        "und erstellen Sie dann eine hilfreiche Endantwort. Verwenden Sie bei der Verwendung eines Tools klare, präzise Anfragen."
    ),
    tools=[plan_weekend, plan_meal],
)

print("✅ Supervisor-Agent erstellt")

## 9. Consulta en Alemán

In [ ]:
async def main():
    print("\n" + "="*80)
    print("ANFRAGE AN DEN SUPERVISOR")
    print("="*80)
    
    user_query = "Was kann ich dieses Wochenende unternehmen und was kann ich zum Mittagessen kochen?"
    print(f"\n👤 Benutzer: {user_query}\n")
    print("─" * 80)
    
    response = await supervisor_agent.run(user_query)
    
    print("\n" + "─" * 80)
    print("\n🤖 ANTWORT DES SUPERVISORS:\n")
    print(response.text)
    print("\n" + "="*80)

await main()

## 10. Más Pruebas en Alemán

In [ ]:
async def test_weekend():
    print("\n" + "="*80)
    print("TEST: NUR WOCHENENDPLANUNG")
    print("="*80 + "\n")
    
    query = "Welche Aktivitäten wären gut für dieses Wochenende in Berlin?"
    print(f"👤 Benutzer: {query}\n")
    print("─" * 80)
    
    response = await supervisor_agent.run(query)
    
    print("\n" + "─" * 80)
    print("\n🤖 Antwort:\n")
    print(response.text)

await test_weekend()

In [ ]:
async def test_meal():
    print("\n" + "="*80)
    print("TEST: NUR MAHLZEITPLANUNG")
    print("="*80 + "\n")
    
    query = "Was soll ich heute Abend kochen? Ich mag Pasta."
    print(f"👤 Benutzer: {query}\n")
    print("─" * 80)
    
    response = await supervisor_agent.run(query)
    
    print("\n" + "─" * 80)
    print("\n🤖 Antwort:\n")
    print(response.text)

await test_meal()

## Conclusión

Este ejemplo demuestra:

### 1. **Soporte Multilingüe**
- El sistema funciona perfectamente en alemán
- Las descripciones de herramientas pueden estar en cualquier idioma
- Los modelos LLM entienden el contexto en múltiples idiomas

### 2. **Misma Arquitectura, Diferente Idioma**
- La estructura del código es idéntica
- Solo cambian las instrucciones y descripciones
- El comportamiento es consistente

### 3. **Aplicaciones Internacionales**
Este enfoque permite:
- Crear agentes para mercados específicos
- Mantener una base de código única
- Adaptar solo el contenido lingüístico

### Idiomas Soportados:
Los modelos de Azure OpenAI soportan múltiples idiomas incluyendo:
- Inglés, Español, Alemán, Francés, Italiano
- Portugués, Holandés, Polaco, Ruso
- Chino, Japonés, Coreano
- Y muchos más...

Puedes adaptar este código a cualquier idioma simplemente traduciendo:
1. Las instrucciones de los agentes
2. Las descripciones de parámetros en `Field(description="...")`
3. Los datos de retorno de las funciones (opcional)